In [194]:
filename=input()

u200.xls


In [195]:
import requests
from bs4 import BeautifulSoup
import re
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

In [196]:
import time
start_time = time.time()

In [197]:
data=pd.read_excel(filename)
# print(data[['linkk']]) 
df=pd.DataFrame(data)
# dfs=df['linkk']
print(df['links'][0])
num_of_links=df.shape[0]
l_names=[]
l_w_name=[]

http://127.0.0.1:8000/p1


In [198]:
def project_ndd1(url):
    r = requests.get(url)# r variable has all the HTML code
    htmlContent = r.content# r returns response so if we want the code we write r.content
    #print(htmlContent)
    soup = BeautifulSoup(r.content, 'html.parser')
    s1=""
    for i in soup.find_all('p'):
        a=(i.text)
        s1+=(i.text)
    # print(a) 
    # s1.strip()  
    while "\n" in s1:
        s1=s1.replace("\n"," ")
    while "     " in s1:
        s1=s1.replace("     "," ")
    while "    " in s1:
        s1=s1.replace("    "," ")    
    punc='''!()-[]{};:'"\,<>./?@#$%^&*_~”’“‘|—•'''
    stop_word=["a","the","an","is"]
    for ele in s1:
        if ele in punc:
            s1 = s1.replace(ele, "")
    no_of_words=list(s1.split())
#     print(len(no_of_words)) 
    s2=""
    for i in no_of_words:
        s2+=i
        s2+=" "
#     print(s2)
    stop_words = stopwords.words('english')
    l=list(s2.split())
    for x in range(len(l)):
        if l[x] in stop_words:
            l[x]=""
    s_inf=""
    for x in l:
        s_inf+=x
        s_inf+=" "     
#     print(s_inf)   
    di={}
    for x in range(len(l)):
        b=l[x].lower()
        if b=="" or b.isnumeric():
            continue
        a=l.count(b)
        di[b]=a
#     print(di)
    def get_value(tp):
        return di[tp] 
    l=[]
    i=1
    for x in sorted(di,key=get_value, reverse=True):
        ll=[]
        ll.append(i)
        ll.append(x)
        ll.append(di[x])
        l.append(ll)
        i+=1
#     print(l)
    #data=pd.DataFrame(l)
    #df=data.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]]
#     ll=[]
#     for i in range(15):
#         ll.append(l[i])
    return l[:15]

In [199]:
#brute force/manual entering
# l1=[]
# while True:
#     ll=[]
#     s=input()
#     if s=="-1":
#         break
# #     ll.append(project_ndd1(s))
#     #l1.append(ll)
#     l1.append(project_ndd1(s))
# print(l1)   

l1=[]
for i in range(num_of_links):
    url=df['links'][i]
    l_w_name.append(f"w{i+1}")
    l1.append(project_ndd1(url))

In [200]:
# print(l1) 
# df=pd.DataFrame(l1)
df.rename(columns={0:'T_IND',1:'WORD',2:'COUNT'},inplace=True)
list_of_dfs=[]
diff_df_t1=[]
diff_df_t2=[]
# print(df) 
for i in range(len(l1)):
    for j in range(i+1,len(l1)):
        if i==j:
            continue
        ll1=[]
        ll2=[]
        ll1_ig=[]
        ll2_ig=[]
        for k in range(len(l1[i])):
            for l in range(len(l1[j])):
                if l1[i][k][1]==l1[j][l][1]:
                    ll1.append(l1[i][k])
                    ll1_ig.append(k)
                    ll2.append(l1[j][l])
                    ll2_ig.append(l)
        df1=pd.DataFrame(ll1)
        df1.rename(columns={0:f'T_IND{i}',1:'WORD',2:'COUNT'},inplace=True)
        df2=pd.DataFrame(ll2)
        df2.rename(columns={0:f'T_IND{j}',1:'WORD',2:'COUNT'},inplace=True)
        frames=[df1,df2]
        result = pd.concat(frames, axis=1, join='inner')
        list_of_dfs.append(result)
        l_diff_t1=[]
        l_diff_t2=[]
        for q in range(len(l1[i])):
            if q not in ll1_ig:
                l_diff_t1.append(l1[i][q])
        for r in range(len(l1[j])):
            if r not in ll2_ig:
                l_diff_t2.append(l1[j][r])  
        df3=pd.DataFrame(l_diff_t1)
        df3.rename(columns={0:'T_IND',1:'WORD',2:'COUNT'},inplace=True)
        df4=pd.DataFrame(l_diff_t2)
        df4.rename(columns={0:'T_IND',1:'WORD',2:'COUNT'},inplace=True)
        diff_df_t1.append(df3)
        diff_df_t2.append(df4) 
        v_1=l_w_name[i]
        v_2=l_w_name[j]
        lll=[v_1,v_2]
        l_names.append(lll)

In [201]:
# print(list_of_dfs[0])
# # df_prime=pd.DataFrame(list_of_dfs[0])
# print(diff_df_t1)
# print(diff_df_t2)

In [202]:
import math
df_incl_sdc=[]
for i in range(len(list_of_dfs)):
    local_df=pd.DataFrame(list_of_dfs[i])
    ll_sdc=[]
    for j in range(len(list_of_dfs[i])):
        val1=math.log(local_df.iat[j,2]/local_df.iat[j,5],10)
        val2=abs(1+local_df.iat[j,0]-local_df.iat[j,3])
        l_sdc=val1*val2
        ll_sdc.append(l_sdc)
    ll_sdc_df=pd.DataFrame(ll_sdc)
    df_ele= pd.concat([local_df,ll_sdc_df], axis=1, join='inner')
    df_ele.rename(columns={0:'SDC_VAL'},inplace=True)
    df_incl_sdc.append(df_ele)

In [203]:
# df_incl_sdc

In [204]:
df_incl_sdt_t1=[]
for i in range(len(diff_df_t1)):
    local_diff=pd.DataFrame(diff_df_t1[i])
    ll_sdt=[]
    alpha= local_diff.shape[0]
    for j in range(alpha):
        if local_diff.iat[j,2]==0:
            continue
        val=math.log(local_diff.iat[j,2],10)*16
        ll_sdt.append(val)
    sdt_df=pd.DataFrame(ll_sdt)
    df_sdt= pd.concat([local_diff,sdt_df], axis=1, join='inner')
    df_sdt.rename(columns={0:'SDT1_VAL'},inplace=True)
    df_incl_sdt_t1.append(df_sdt)

In [205]:
# df_incl_sdt_t1

In [206]:
df_incl_sdt_t2=[]
for i in range(len(diff_df_t2)):
    local_diff=pd.DataFrame(diff_df_t2[i])
    ll_sdt=[]
    alpha= local_diff.shape[0]
    for j in range(alpha):
#         print(local_diff.iat[j,2])
        val=math.log(abs(local_diff.iat[j,2])+1,10)*16
        ll_sdt.append(val)
    sdt_df=pd.DataFrame(ll_sdt)
    df_sdt= pd.concat([local_diff,sdt_df], axis=1, join='inner')
    df_sdt.rename(columns={0:'SDT2_VAL'},inplace=True)
    df_incl_sdt_t2.append(df_sdt)

In [207]:
# df_incl_sdt_t2

In [208]:
# df_incl_sdt_t2[9]['SDT2_VAL'].sum()

In [209]:
count_of_unique=0
count_of_ndd=0
for i in range(len(df_incl_sdc)):
    df_1=pd.DataFrame(df_incl_sdc[i])
    df_2=pd.DataFrame(df_incl_sdt_t1[i])
    df_3=pd.DataFrame(df_incl_sdt_t2[i])
    if df_1.empty and not df_3.empty and not df_2.empty:
#         print(df_3)
#         print(df_2['SDT1_VAL'])
        ssm=df_2['SDT1_VAL'].sum()+df_3['SDT2_VAL'].sum()
    
    elif df_2.empty and not df_3.empty and not df_1.empty:
#         print(df_3)
        ssm=df_1['SDC_VAL'].sum()+df_3['SDT2_VAL'].sum()
    
    elif df_3.empty and not df_2.empty and not df_1.empty:
#         print(df_3)
        ssm=df_1['SDC_VAL'].sum()+df_2['SDT1_VAL'].sum()
    
    elif df_2.empty and df_3.empty:
        ssm=df_1['SDC_VAL'].sum()
        
    elif df_1.empty and df_2.empty:
        ssm=df_3['SDT2_VAL'].sum()
        
    elif df_1.empty and df_3.empty:
        ssm=df_2['SDT1_VAL'].sum()
        
    else:
#         print(df_3)
        ssm=df_1['SDC_VAL'].sum()+df_2['SDT1_VAL'].sum()+df_3['SDT2_VAL'].sum()
    ssm/=15
    print("AFTER CALCULATION THE SSM IS {} for the datasets {}".format(ssm,l_names[i]))
    if ssm<19.50:
        print("NDD")
        count_of_ndd+=1
    elif ssm>=19.50:
        print("UNIQUE")
        count_of_unique+=1

AFTER CALCULATION THE SSM IS 31.980208285438582 for the datasets ['w1', 'w2']
UNIQUE
AFTER CALCULATION THE SSM IS 33.89947708263082 for the datasets ['w1', 'w3']
UNIQUE
AFTER CALCULATION THE SSM IS 30.24607315637724 for the datasets ['w1', 'w4']
UNIQUE
AFTER CALCULATION THE SSM IS 31.573131167790425 for the datasets ['w1', 'w5']
UNIQUE
AFTER CALCULATION THE SSM IS 32.06278301835751 for the datasets ['w1', 'w6']
UNIQUE
AFTER CALCULATION THE SSM IS 32.62923452572399 for the datasets ['w1', 'w7']
UNIQUE
AFTER CALCULATION THE SSM IS 37.972959387127645 for the datasets ['w1', 'w8']
UNIQUE
AFTER CALCULATION THE SSM IS 32.10723683534437 for the datasets ['w1', 'w9']
UNIQUE
AFTER CALCULATION THE SSM IS 35.61034301367375 for the datasets ['w1', 'w10']
UNIQUE
AFTER CALCULATION THE SSM IS 27.293867509771818 for the datasets ['w1', 'w11']
UNIQUE
AFTER CALCULATION THE SSM IS 34.54485605055443 for the datasets ['w1', 'w12']
UNIQUE
AFTER CALCULATION THE SSM IS 33.114887862111985 for the datasets ['w1

AFTER CALCULATION THE SSM IS 25.55686191786699 for the datasets ['w2', 'w139']
UNIQUE
AFTER CALCULATION THE SSM IS 29.919448738393815 for the datasets ['w2', 'w140']
UNIQUE
AFTER CALCULATION THE SSM IS 25.82729231587745 for the datasets ['w2', 'w141']
UNIQUE
AFTER CALCULATION THE SSM IS 26.045088993983594 for the datasets ['w2', 'w142']
UNIQUE
AFTER CALCULATION THE SSM IS 26.324368306478686 for the datasets ['w2', 'w143']
UNIQUE
AFTER CALCULATION THE SSM IS 24.989199933023382 for the datasets ['w2', 'w144']
UNIQUE
AFTER CALCULATION THE SSM IS 26.785851286321837 for the datasets ['w2', 'w145']
UNIQUE
AFTER CALCULATION THE SSM IS 30.473731116001147 for the datasets ['w2', 'w146']
UNIQUE
AFTER CALCULATION THE SSM IS 31.179375027442255 for the datasets ['w2', 'w147']
UNIQUE
AFTER CALCULATION THE SSM IS 30.098099433680606 for the datasets ['w2', 'w148']
UNIQUE
AFTER CALCULATION THE SSM IS 28.795936628996596 for the datasets ['w2', 'w149']
UNIQUE
AFTER CALCULATION THE SSM IS 26.4815843575867

UNIQUE
AFTER CALCULATION THE SSM IS 33.31437685758372 for the datasets ['w4', 'w34']
UNIQUE
AFTER CALCULATION THE SSM IS 36.78270195659498 for the datasets ['w4', 'w35']
UNIQUE
AFTER CALCULATION THE SSM IS 41.389894636056525 for the datasets ['w4', 'w36']
UNIQUE
AFTER CALCULATION THE SSM IS 39.582129638274544 for the datasets ['w4', 'w37']
UNIQUE
AFTER CALCULATION THE SSM IS 30.390273082436526 for the datasets ['w4', 'w38']
UNIQUE
AFTER CALCULATION THE SSM IS 33.12902851966856 for the datasets ['w4', 'w39']
UNIQUE
AFTER CALCULATION THE SSM IS 40.93103805062591 for the datasets ['w4', 'w40']
UNIQUE
AFTER CALCULATION THE SSM IS 27.356098489516857 for the datasets ['w4', 'w41']
UNIQUE
AFTER CALCULATION THE SSM IS 32.33937557836353 for the datasets ['w4', 'w42']
UNIQUE
AFTER CALCULATION THE SSM IS 36.01843401827447 for the datasets ['w4', 'w43']
UNIQUE
AFTER CALCULATION THE SSM IS 28.71735149921078 for the datasets ['w4', 'w44']
UNIQUE
AFTER CALCULATION THE SSM IS 38.55912026346644 for the

AFTER CALCULATION THE SSM IS 27.011426330530558 for the datasets ['w5', 'w125']
UNIQUE
AFTER CALCULATION THE SSM IS 28.051427497752517 for the datasets ['w5', 'w126']
UNIQUE
AFTER CALCULATION THE SSM IS 27.88309119923953 for the datasets ['w5', 'w127']
UNIQUE
AFTER CALCULATION THE SSM IS 31.393664094432044 for the datasets ['w5', 'w128']
UNIQUE
AFTER CALCULATION THE SSM IS 25.29479850051627 for the datasets ['w5', 'w129']
UNIQUE
AFTER CALCULATION THE SSM IS 26.09106923888644 for the datasets ['w5', 'w130']
UNIQUE
AFTER CALCULATION THE SSM IS 23.732654838539407 for the datasets ['w5', 'w131']
UNIQUE
AFTER CALCULATION THE SSM IS 27.845844334985063 for the datasets ['w5', 'w132']
UNIQUE
AFTER CALCULATION THE SSM IS 30.602939311470244 for the datasets ['w5', 'w133']
UNIQUE
AFTER CALCULATION THE SSM IS 30.410071037286592 for the datasets ['w5', 'w134']
UNIQUE
AFTER CALCULATION THE SSM IS 26.074314153613614 for the datasets ['w5', 'w135']
UNIQUE
AFTER CALCULATION THE SSM IS 27.37970821983852

AFTER CALCULATION THE SSM IS 36.00200738455185 for the datasets ['w7', 'w83']
UNIQUE
AFTER CALCULATION THE SSM IS 34.21501512378312 for the datasets ['w7', 'w84']
UNIQUE
AFTER CALCULATION THE SSM IS 36.11217756454359 for the datasets ['w7', 'w85']
UNIQUE
AFTER CALCULATION THE SSM IS 37.93289090702626 for the datasets ['w7', 'w86']
UNIQUE
AFTER CALCULATION THE SSM IS 34.79415861136268 for the datasets ['w7', 'w87']
UNIQUE
AFTER CALCULATION THE SSM IS 33.63734229354104 for the datasets ['w7', 'w88']
UNIQUE
AFTER CALCULATION THE SSM IS 36.06905565984231 for the datasets ['w7', 'w89']
UNIQUE
AFTER CALCULATION THE SSM IS 33.104949354944644 for the datasets ['w7', 'w90']
UNIQUE
AFTER CALCULATION THE SSM IS 32.07439306806667 for the datasets ['w7', 'w91']
UNIQUE
AFTER CALCULATION THE SSM IS 33.25532940593269 for the datasets ['w7', 'w92']
UNIQUE
AFTER CALCULATION THE SSM IS 35.01532747037694 for the datasets ['w7', 'w93']
UNIQUE
AFTER CALCULATION THE SSM IS 34.00052202433821 for the datasets 

...

AFTER CALCULATION THE SSM IS 24.335435818549588 for the datasets ['w151', 'w175']
UNIQUE
AFTER CALCULATION THE SSM IS 34.46716994166472 for the datasets ['w151', 'w176']
UNIQUE
AFTER CALCULATION THE SSM IS 20.23616655266901 for the datasets ['w151', 'w177']
UNIQUE
AFTER CALCULATION THE SSM IS 19.250985682511327 for the datasets ['w151', 'w178']
NDD
AFTER CALCULATION THE SSM IS 19.669072409032438 for the datasets ['w151', 'w179']
UNIQUE
AFTER CALCULATION THE SSM IS 22.377236312411096 for the datasets ['w151', 'w180']
UNIQUE
AFTER CALCULATION THE SSM IS 22.302549655771642 for the datasets ['w151', 'w181']
UNIQUE
AFTER CALCULATION THE SSM IS 21.654931359619283 for the datasets ['w151', 'w182']
UNIQUE
AFTER CALCULATION THE SSM IS 21.64534948470544 for the datasets ['w151', 'w183']
UNIQUE
AFTER CALCULATION THE SSM IS 21.718581662442684 for the datasets ['w151', 'w184']
UNIQUE
AFTER CALCULATION THE SSM IS 19.093125300529515 for the datasets ['w151', 'w185']
NDD
AFTER CALCULATION THE SSM IS 2

AFTER CALCULATION THE SSM IS 14.937627465683597 for the datasets ['w157', 'w182']
NDD
AFTER CALCULATION THE SSM IS 20.06754758405858 for the datasets ['w157', 'w183']
UNIQUE
AFTER CALCULATION THE SSM IS 20.381987524476237 for the datasets ['w157', 'w184']
UNIQUE
AFTER CALCULATION THE SSM IS 17.009581924436205 for the datasets ['w157', 'w185']
NDD
AFTER CALCULATION THE SSM IS 18.26722574305345 for the datasets ['w157', 'w186']
NDD
AFTER CALCULATION THE SSM IS 23.68898014134074 for the datasets ['w157', 'w187']
UNIQUE
AFTER CALCULATION THE SSM IS 22.51966300924661 for the datasets ['w157', 'w188']
UNIQUE
AFTER CALCULATION THE SSM IS 19.551398827950546 for the datasets ['w157', 'w189']
UNIQUE
AFTER CALCULATION THE SSM IS 20.485182745618012 for the datasets ['w157', 'w190']
UNIQUE
AFTER CALCULATION THE SSM IS 26.797059347893075 for the datasets ['w157', 'w191']
UNIQUE
AFTER CALCULATION THE SSM IS 26.74843846775719 for the datasets ['w157', 'w192']
UNIQUE
AFTER CALCULATION THE SSM IS 18.446

AFTER CALCULATION THE SSM IS 19.321540879367493 for the datasets ['w160', 'w171']
NDD
AFTER CALCULATION THE SSM IS 23.526860088845513 for the datasets ['w160', 'w172']
UNIQUE
AFTER CALCULATION THE SSM IS 36.9208949794785 for the datasets ['w160', 'w173']
UNIQUE
AFTER CALCULATION THE SSM IS 28.53597471438226 for the datasets ['w160', 'w174']
UNIQUE
AFTER CALCULATION THE SSM IS 23.087837315918264 for the datasets ['w160', 'w175']
UNIQUE
AFTER CALCULATION THE SSM IS 38.21157840959111 for the datasets ['w160', 'w176']
UNIQUE
AFTER CALCULATION THE SSM IS 21.229639149682157 for the datasets ['w160', 'w177']
UNIQUE
AFTER CALCULATION THE SSM IS 20.28284543924993 for the datasets ['w160', 'w178']
UNIQUE
AFTER CALCULATION THE SSM IS 16.072829997552216 for the datasets ['w160', 'w179']
NDD
AFTER CALCULATION THE SSM IS 20.99468985483833 for the datasets ['w160', 'w180']
UNIQUE
AFTER CALCULATION THE SSM IS 23.57704059302878 for the datasets ['w160', 'w181']
UNIQUE
AFTER CALCULATION THE SSM IS 18.73

AFTER CALCULATION THE SSM IS 27.27538602210155 for the datasets ['w168', 'w175']
UNIQUE
AFTER CALCULATION THE SSM IS 38.3462169972346 for the datasets ['w168', 'w176']
UNIQUE
AFTER CALCULATION THE SSM IS 25.7391276842127 for the datasets ['w168', 'w177']
UNIQUE
AFTER CALCULATION THE SSM IS 24.728914419654174 for the datasets ['w168', 'w178']
UNIQUE
AFTER CALCULATION THE SSM IS 27.796634530287193 for the datasets ['w168', 'w179']
UNIQUE
AFTER CALCULATION THE SSM IS 28.196144265499303 for the datasets ['w168', 'w180']
UNIQUE
AFTER CALCULATION THE SSM IS 30.74191885629256 for the datasets ['w168', 'w181']
UNIQUE
AFTER CALCULATION THE SSM IS 28.0572937518266 for the datasets ['w168', 'w182']
UNIQUE
AFTER CALCULATION THE SSM IS 25.3725776666892 for the datasets ['w168', 'w183']
UNIQUE
AFTER CALCULATION THE SSM IS 27.740977172465577 for the datasets ['w168', 'w184']
UNIQUE
AFTER CALCULATION THE SSM IS 24.571811422821057 for the datasets ['w168', 'w185']
UNIQUE
AFTER CALCULATION THE SSM IS 31

AFTER CALCULATION THE SSM IS 18.885041207138816 for the datasets ['w179', 'w189']
NDD
AFTER CALCULATION THE SSM IS 20.92951969557408 for the datasets ['w179', 'w190']
UNIQUE
AFTER CALCULATION THE SSM IS 23.112245172806684 for the datasets ['w179', 'w191']
UNIQUE
AFTER CALCULATION THE SSM IS 20.74857896289628 for the datasets ['w179', 'w192']
UNIQUE
AFTER CALCULATION THE SSM IS 19.21224037232303 for the datasets ['w179', 'w193']
NDD
AFTER CALCULATION THE SSM IS 23.599588007787386 for the datasets ['w179', 'w194']
UNIQUE
AFTER CALCULATION THE SSM IS 25.04315753342871 for the datasets ['w179', 'w195']
UNIQUE
AFTER CALCULATION THE SSM IS 20.973095056539552 for the datasets ['w179', 'w196']
UNIQUE
AFTER CALCULATION THE SSM IS 19.269840540864575 for the datasets ['w179', 'w197']
NDD
AFTER CALCULATION THE SSM IS 21.155234274401664 for the datasets ['w179', 'w198']
UNIQUE
AFTER CALCULATION THE SSM IS 18.67494084175382 for the datasets ['w179', 'w199']
NDD
AFTER CALCULATION THE SSM IS 21.500454

In [210]:
print("the number of unique pairs is: ",count_of_unique)
print("the number of ndd pairs is: ",count_of_ndd)

the number of unique pairs is:  19402
the number of ndd pairs is:  498


In [211]:
with open("res1.txt","a") as f:
    s1=f"""{count_of_unique} """
    s2=f"""{count_of_ndd} """
    s3=f"""{filename} """
    f.write("unstemmed ")
    f.write(s3)
    f.write(s1)
    f.write(s2)
    f.write("%s seconds\n" % (time.time() - start_time))

In [212]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 116.67142653465271 seconds ---


In [213]:
print(filename)

u200.xls
